## Read and visualize simulation data for two-dimensional flow around a pitching NACA0015 airfoil at $Re = 1000$.

### Airfoil kinematics: $\theta = \theta_0 + \Delta \theta sin(2\pi f^*t)$, where $f^* = 0.15$, $\Delta \theta = 25^{\circ}$, $\theta_0 = 15^{\circ}$.
***
***

### Function: Read the coordinates of the Cartesian grid from the simulation data
The grid used for the simulation is stored in `xgrid.dat` and `ygrid.dat` files for X and Y directions

In [6]:
using DelimitedFiles
function set_grid()
    global overall_path
    # Read grid files
    gridfile = overall_path * "xgrid.dat"
    x_grid = readdlm(gridfile)[:,2]
    gridfile = overall_path * "ygrid.dat"
    y_grid = readdlm(gridfile)[:,2]
    # Create arrays for cell-centered grid points
    nxc = size(x_grid, 1) - 1
    nyc = size(y_grid, 1) - 1
    xc = zeros(nxc)
    yc = zeros(nyc)
    # Convert face-centered to cell-centered coordinates
    for i in 1:nxc
        xc[i] = 0.5 * (x_grid[i] + x_grid[i+1])
    end
    for j in 1:nyc
        yc[j] = 0.5 * (y_grid[j] + y_grid[j+1])
    end
    return nxc, nyc, xc, yc
end;

### Function: Read data within a rectangular subdomain inside the cartesian grid.

>#### Function arguments:
`data_path`: Location of simulation data files (`qout*.dat` data files)<br>
`tstep`: Timestep to read<br>
`varlist`: Indices of variables to read (depends on the format of the data file being read)<br>
`nxc`, `nyc`: Number of grid points in the X and Y directions (cell-centered)<br>
`xstart`, `xend`, `ystart`, `yend`: Start and end indices of the "box" to be read within the grid in the X and Y direction

In [7]:
function read_subdomain_data(data_path,tstep,varlist,nxc,nyc,xstart,xend,ystart,yend)
    filename = data_path * "qout." * lpad(Int(tstep),7,'0') * ".dat"
    f = open(filename,"r")
    # Vector to store all the variables read in a flattened representation
    result = []
    # Iterate through list of variables to read
    for varnum in varlist
        # Move to the start of each variable in the binary file
        seek(f,8*nxc*nyc*(varnum-1))
        # Read nxc*nyc entries corresponding to the current variable on the grid
        fullfield = reinterpret(Float64, read(f, 8*nxc*nyc))
        # Reshape the flattened array read above to a 2D matrix
        fullfield_2d = reshape(fullfield,(nxc,nyc))
        # Store the part of the grid contained within the box [xstart,xend],[ystart,yend]
        field_subdomain = fullfield_2d[xstart:xend-1,ystart:yend-1]
        # Flatten and store the variable within the "box"/subdomain of the grid
        result = vcat(result,vec(field_subdomain))
    end
    close(f)
    return result
end;

### Function: Plot 2D contours of specified variables

>#### Function arguments:
`save_path`: Directory to save figure into<br>
`var`: Data to be plotted, represented as a flattened array<br>
`x_contour`, `y_contour`: X and Y coordinates corresponding to the "box"/subdomain of the grid being plotted<br>
`levels`: Contour levels<br>
`xl`, `yl`: X and Y contour limits

In [26]:
using PyPlot

function plot_contour(save_path,var,varname,x_contour,y_contour,levels,xl,yl)
    my_dpi = 100
    # Convert flattened array to 2D matrix
    var_contour = transpose(reshape(var,(length(x_contour)-1,length(y_contour)-1)))
    # Plot contour
    CS = contourf(x_contour[1:end-1],y_contour[1:end-1],var_contour,levels=levels,cmap=ColorMap("RdBu_r"),extend="both")
    gca().set_aspect("equal")
    xlim(xl)
    ylim(yl)
    title(varname)
    colorbar()
    savefig(joinpath(overall_path, save_path, "New", "vort", save_path * "_" * varname * ".png"), dpi=my_dpi)
    close()
end;

-----------
***

#### Set up the Cartesian grid


In [9]:
overall_path = "/Users/jjser/Desktop/Julia_FTLE_Karthik_Data/airfoil2D_f0.15_amp25deg_KM/"
#"/Users/karthik-macbook/GDrive-local/AIAA_review_paper/airfoil2D_f0.15_amp25deg_KM/"

nxc,nyc,xc,yc = set_grid();

#### Specify the timesteps that need to be read, and the size of the subdomain to read

In [10]:
# Timesteps to read from qout.*.dat files
tstep_start = 200 
tstep_end = 150000
tstep_delta = 200

num_tsteps = Int64((tstep_end-tstep_start)/tstep_delta + 1)

tstep_range = collect(range(tstep_start, tstep_end, length=num_tsteps))

# Spatial extent of a rectangular "box"/subdomain within the Cartesian grid to read
x_start_loc = 9.5
x_end_loc = 12.0
y_start_loc = 9.0
y_end_loc = 11.0

# Find indices in the grid corresponding to above X and Y coordinates
x_start = argmin(abs.(xc.-x_start_loc))
x_end = argmin(abs.(xc.-x_end_loc))+1
y_start = argmin(abs.(yc.-y_start_loc))
y_end = argmin(abs.(yc.-y_end_loc))+1

# Number of grid points in the "box"/subdomain within the Cartesian grid to read
nx_subdomain = x_end-x_start
ny_subdomain = y_end-y_start
subdomain_size = nx_subdomain*ny_subdomain;

#### Specify the indices of the variables that need to be read and their names

In [11]:
var_list = [1,2,3,4,5,10,13];
var_names = ["u","v","p","iblank","vorticity","fpm_scalar","vif"];

#### Read the data corresponding to the timesteps and variables specified above

In [12]:
# Number of variables
numvar = length(var_list)

# Dictionary associating the order of variable names with indices
var_idxs = Dict(zip(var_names,1:numvar))

# Data object to store all variables at all timesteps
# For each timestep, all variables are stored sequentially as flattened arrays
data = zeros(nx_subdomain*ny_subdomain*numvar,num_tsteps)

for (i,tstep) in enumerate(tstep_range)
    data_subdomain = read_subdomain_data(overall_path,tstep,var_list,nxc,nyc,x_start,x_end,y_start,y_end)
    data[:,i] = data_subdomain[:]
end

#### Create and save contour plots for specified variables

In [21]:
subfolder = "plots_JJ"
for (i,tstep) in enumerate(tstep_range)    
    # Specify the variable to plot (should be a member of `var_names`)
    var_name = "v"
    var_id = var_idxs[var_name]
    plot_contour(subfolder,data[(var_id-1)*subdomain_size+1:var_id*subdomain_size,i],var_name*"_"*lpad(Int(tstep),7,'0'),xc[x_start:x_end],yc[y_start:y_end],range(-1.0,1.0,length=30),[x_start_loc,x_end_loc],[y_start_loc,y_end_loc])
end

In [18]:
im = load("/Users/jjser/Desktop/Julia_FTLE_Karthik_Data/airfoil2D_f0.15_amp25deg_KM/plots_JJ/New/u/plots_JJ_u_0000200.png")
println(size(im))

(480, 640)


In [31]:
gif_arr = ones(RGB{N0f8}, (480, 640, num_tsteps)) 
save_path = "plots_JJ"
for tstep in 1:num_tsteps-1
    time = Int(tstep_start+(tstep-1)*tstep_delta)
    varname = lpad(time,7,'0')
    im = load(joinpath(overall_path, save_path, "New", "v", save_path * "_v_" * varname * ".png"))
    gif_arr[:,:,tstep] = im
end # for 

save(joinpath(overall_path, save_path, "New", "vgif.gif"), gif_arr)
#println("done")